This jupyter nodebook is for RQ2 of the project to measure what is the minimum required data of SO dataset to achieve acceptable results compared to maximum data size. I start from 10% of the total amount of data to 100%.

# Load data

In [108]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/SE\ project/dataset/EMTK_datasets-v1.0.zip

In [110]:
import pandas as pd
df = pd.read_excel('/content/collab-uniba-EMTK_datasets-c200f78/so/emotions/Emotions_GoldSandard_andAnnotation.xlsx',sheet_name=None)

In [111]:
df

{'Anger_all':      Group     Set  Unnamed: 2  ... Gold Label Unnamed: 8 Unnamed: 9
 0        A  Second           1  ...        NaN        NaN        NaN
 1        A  Second           2  ...        NaN        NaN        NaN
 2        A  Second           3  ...        NaN        NaN        NaN
 3        A  Second           4  ...        NaN        NaN        NaN
 4        A  Second           5  ...        NaN        NaN        NaN
 ...    ...     ...         ...  ...        ...        ...        ...
 4795     D   Third         496  ...        NaN        NaN        NaN
 4796     D   Third         497  ...        NaN        NaN        NaN
 4797     D   Third         498  ...        NaN        NaN        NaN
 4798     D   Third         499  ...        NaN        NaN        NaN
 4799     D   Third         500  ...        NaN        NaN        NaN
 
 [4800 rows x 10 columns],
 'Fear_all':      Group     Set  Unnamed: 2  ... Gold Label Unnamed: 8 Unnamed: 9
 0        A  Second           1  ...

In [112]:
df["Love_all"]

,Group,Set,Unnamed: 2,Text,rater 1,rater 2,rater 3,Gold Label,Unnamed: 8,Unnamed: 9
0,A,Second,1,SVG transform on text attribute works excellen...,X,NaN,X,LOVE,NaN,NaN
1,A,Second,2,Excellent! This is exactly what I needed. Thanks!,X,x,X,LOVE,NaN,NaN
2,A,Second,3,Have added a modern solution as of May 2014 in...,NaN,NaN,NaN,NaN,NaN,NaN
3,A,Second,4,Have you tried removing 'preload' attribute? (...,NaN,NaN,NaN,NaN,NaN,NaN
4,A,Second,5,"A smarter, entirely C++-way of doing what you ...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4795,D,Third,496,Yes - that feature is extremely useful for wri...,NaN,NaN,x,NaN,NaN,NaN
4796,D,Third,497,"Works great! And you can add ""desc"" after the ...",x,NaN,x,LOVE,NaN,NaN
4797,D,Third,498,"Yeah, I didn't know about the non-greedy thing...",x,NaN,NaN,NaN,NaN,NaN
4798,D,Third,499,Fortunately I'm doing *very* little with Offic...,x,X,x,LOVE,NaN,NaN


In [113]:
text = []
neutral_emotional = []
label_sets = []

for i in range(4800):
  
  label = 0
  label_set = []
  for sheet in ['Love_all', 'Joy_all', 'Anger_all', 'Sadness_all', 'Fear_all', 'Surprise_all']:
    gold_label = df[sheet].loc[i].iat[7]
    if gold_label in ['LOVE', 'JOY', 'ANGER', 'SADNESS', 'FEAR', 'SURPRISE']:
      label = 1
      label_set.append(gold_label)
  if not 'SURPRISE' in label_set:
    neutral_emotional.append(label)
    label_sets.append(label_set)
    text.append(df['Love_all'].loc[i].iat[3])
  

In [114]:
def not_both(label_set,i):
  positive = ['LOVE','JOY']
  negative = ['SADNESS', 'FEAR', 'ANGER']
  is_positive = False
  is_negative = False
  for label in label_set:
    if label in positive:
      is_positive = True
    elif label in negative:
      is_negative = True
  if is_negative and is_positive:
    return False
  else:
    return True

def find_semantic(label_set):
  positive = ['LOVE','JOY']
  negative = ['SADNESS', 'FEAR', 'ANGER']
  if label_set[0] in positive:
    return 1
  return 0

In [115]:
text_semantic = []
semantic = []

for i in range(4755):
  if neutral_emotional[i]:
    if not_both(label_sets[i],i):
      text_semantic.append(text[i])
      semantic.append(find_semantic(label_sets[i]))


In [116]:
text_all = []
three_cat = []

for i in range(4755):
  if neutral_emotional[i]==0:
    text_all.append(text[i])
    three_cat.append('Neutral')
for i in range(len(text_semantic)):
  text_all.append(text_semantic[i])
  if semantic[i]:
    three_cat.append('Positive')
  else:
    three_cat.append('Negative')

In [117]:
len(text_all)

4730

# Model setting

In [118]:
import io
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM, Bidirectional
from tensorflow.keras.layers import TextVectorization
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import LSTM, GRU
from keras.layers import Flatten, Dense
from keras.callbacks import ModelCheckpoint
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing. sequence import pad_sequences

In [119]:
# define example
data = three_cat
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)

['Neutral' 'Neutral' 'Neutral' ... 'Positive' 'Positive' 'Positive']
[1 1 1 ... 2 2 2]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [120]:
len(text_all)*0.7

3311.0

In [121]:
history_GRU  = []

# 10 %

In [122]:
rate = 0.1

In [123]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [124]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [125]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [126]:
d= 60

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(GRU(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [127]:
# fit the model
mcp_save = ModelCheckpoint('model_best.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/10
42/42 [==============================] - 9s 168ms/step - loss: 1.0730 - acc: 0.4411 - val_loss: 1.0187 - val_acc: 0.4637
Epoch 2/10
42/42 [==============================] - 4s 91ms/step - loss: 0.9297 - acc: 0.5801 - val_loss: 0.9402 - val_acc: 0.5821
Epoch 3/10
42/42 [==============================] - 4s 88ms/step - loss: 0.8415 - acc: 0.6435 - val_loss: 0.8983 - val_acc: 0.6047
Epoch 4/10
42/42 [==============================] - 4s 88ms/step - loss: 0.7826 - acc: 0.6586 - val_loss: 0.8918 - val_acc: 0.6166
Epoch 5/10
42/42 [==============================] - 4s 88ms/step - loss: 0.7244 - acc: 0.6858 - val_loss: 0.8967 - val_acc: 0.6314
Epoch 6/10
42/42 [==============================] - 4s 87ms/step - loss: 0.6837 - acc: 0.7009 - val_loss: 0.8524 - val_acc: 0.6519
Epoch 7/10
42/42 [==============================] - 4s 86ms/step - loss: 0.6640 - acc: 0.7221 - val_loss: 0.8506 - val_acc: 0.6378
Epoch 8/10
42/42 [==============================] - 4s 88ms/step - loss: 0.6017 - 

In [128]:
from keras.models import load_model
model = load_model('/content/model_best.hdf5')

In [129]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [130]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_GRU.append(accu)

# 20 %

In [131]:
rate = 0.2

In [132]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [133]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [134]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [135]:
d= 60

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(GRU(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [137]:
# fit the model
mcp_save = ModelCheckpoint('model_best_2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/10
83/83 [==============================] - 6s 71ms/step - loss: 0.4604 - acc: 0.8308 - val_loss: 0.6089 - val_acc: 0.7639
Epoch 2/10
83/83 [==============================] - 6s 67ms/step - loss: 0.4123 - acc: 0.8384 - val_loss: 0.5695 - val_acc: 0.7872
Epoch 3/10
83/83 [==============================] - 7s 85ms/step - loss: 0.3978 - acc: 0.8233 - val_loss: 0.5960 - val_acc: 0.7879
Epoch 4/10
83/83 [==============================] - 9s 105ms/step - loss: 0.3672 - acc: 0.8565 - val_loss: 0.5938 - val_acc: 0.7815
Epoch 5/10
83/83 [==============================] - 10s 123ms/step - loss: 0.3485 - acc: 0.8852 - val_loss: 0.7080 - val_acc: 0.7308
Epoch 6/10
83/83 [==============================] - 9s 110ms/step - loss: 0.3002 - acc: 0.8852 - val_loss: 0.6449 - val_acc: 0.7731
Epoch 7/10
83/83 [==============================] - 9s 110ms/step - loss: 0.2994 - acc: 0.8852 - val_loss: 0.6277 - val_acc: 0.7773
Epoch 8/10
83/83 [==============================] - 10s 120ms/step - loss: 0.2

In [138]:
from keras.models import load_model
model = load_model('/content/model_best_2.hdf5')

In [139]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [140]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_GRU.append(accu)

# 30 %

In [141]:
rate = 0.3

In [142]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [143]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [144]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [145]:
d= 60

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(GRU(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [146]:
# fit the model
mcp_save = ModelCheckpoint('model_best_3.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/10
125/125 [==============================] - 11s 67ms/step - loss: 0.9976 - acc: 0.5196 - val_loss: 0.9871 - val_acc: 0.5236
Epoch 2/10
125/125 [==============================] - 8s 63ms/step - loss: 0.8363 - acc: 0.6485 - val_loss: 0.9052 - val_acc: 0.6180
Epoch 3/10
125/125 [==============================] - 8s 62ms/step - loss: 0.7687 - acc: 0.6677 - val_loss: 0.7631 - val_acc: 0.6779
Epoch 4/10
125/125 [==============================] - 8s 63ms/step - loss: 0.6972 - acc: 0.7080 - val_loss: 0.7633 - val_acc: 0.7026
Epoch 5/10
125/125 [==============================] - 8s 62ms/step - loss: 0.6566 - acc: 0.7311 - val_loss: 0.7505 - val_acc: 0.6808
Epoch 6/10
125/125 [==============================] - 8s 62ms/step - loss: 0.5950 - acc: 0.7694 - val_loss: 0.6462 - val_acc: 0.7322
Epoch 7/10
125/125 [==============================] - 8s 64ms/step - loss: 0.5479 - acc: 0.7724 - val_loss: 0.6379 - val_acc: 0.7357
Epoch 8/10
125/125 [==============================] - 8s 63ms/step -

In [147]:
from keras.models import load_model
model = load_model('/content/model_best_3.hdf5')

In [148]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [149]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_GRU.append(accu)

# 40 %

In [150]:
rate = 0.4

In [151]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [152]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [153]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [154]:
d= 60

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(GRU(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [155]:
# fit the model
mcp_save = ModelCheckpoint('model_best_4.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/10
166/166 [==============================] - 12s 60ms/step - loss: 0.9715 - acc: 0.5249 - val_loss: 0.8669 - val_acc: 0.6209
Epoch 2/10
166/166 [==============================] - 10s 58ms/step - loss: 0.8308 - acc: 0.6276 - val_loss: 0.8654 - val_acc: 0.6286
Epoch 3/10
166/166 [==============================] - 10s 59ms/step - loss: 0.7391 - acc: 0.6737 - val_loss: 0.6708 - val_acc: 0.7315
Epoch 4/10
166/166 [==============================] - 9s 57ms/step - loss: 0.6603 - acc: 0.7228 - val_loss: 0.6611 - val_acc: 0.7407
Epoch 5/10
166/166 [==============================] - 10s 58ms/step - loss: 0.5799 - acc: 0.7621 - val_loss: 0.6580 - val_acc: 0.7449
Epoch 6/10
166/166 [==============================] - 10s 59ms/step - loss: 0.5394 - acc: 0.7908 - val_loss: 0.5475 - val_acc: 0.7949
Epoch 7/10
166/166 [==============================] - 10s 58ms/step - loss: 0.4890 - acc: 0.7938 - val_loss: 0.5458 - val_acc: 0.7914
Epoch 8/10
166/166 [==============================] - 10s 60ms/

In [156]:
from keras.models import load_model
model = load_model('/content/model_best_4.hdf5')

In [157]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [158]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_GRU.append(accu)

# 50 %

In [159]:
rate = 0.5

In [160]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [161]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [162]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [163]:
d= 60

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(GRU(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [164]:
# fit the model
mcp_save = ModelCheckpoint('model_best_5.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/10
207/207 [==============================] - 15s 62ms/step - loss: 0.9787 - acc: 0.5329 - val_loss: 0.8628 - val_acc: 0.5955
Epoch 2/10
207/207 [==============================] - 13s 62ms/step - loss: 0.8140 - acc: 0.6417 - val_loss: 0.7131 - val_acc: 0.6998
Epoch 3/10
207/207 [==============================] - 13s 62ms/step - loss: 0.6948 - acc: 0.6991 - val_loss: 0.6604 - val_acc: 0.7357
Epoch 4/10
207/207 [==============================] - 13s 61ms/step - loss: 0.6228 - acc: 0.7468 - val_loss: 0.6257 - val_acc: 0.7625
Epoch 5/10
207/207 [==============================] - 13s 62ms/step - loss: 0.5539 - acc: 0.7843 - val_loss: 0.5721 - val_acc: 0.7865
Epoch 6/10
207/207 [==============================] - 13s 62ms/step - loss: 0.5139 - acc: 0.8042 - val_loss: 0.6358 - val_acc: 0.7498
Epoch 7/10
207/207 [==============================] - 13s 61ms/step - loss: 0.4874 - acc: 0.8030 - val_loss: 0.5592 - val_acc: 0.7914
Epoch 8/10
207/207 [==============================] - 13s 61ms

In [165]:
from keras.models import load_model
model = load_model('/content/model_best_5.hdf5')

In [166]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [167]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_GRU.append(accu)

# 60 %

In [168]:
rate = 0.6

In [169]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [170]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [171]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [172]:
d= 60

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(GRU(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [173]:
# fit the model
mcp_save = ModelCheckpoint('model_best_6.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/10
249/249 [==============================] - 16s 57ms/step - loss: 0.9588 - acc: 0.5448 - val_loss: 0.8431 - val_acc: 0.6385
Epoch 2/10
249/249 [==============================] - 14s 56ms/step - loss: 0.8006 - acc: 0.6571 - val_loss: 0.6689 - val_acc: 0.7188
Epoch 3/10
249/249 [==============================] - 14s 58ms/step - loss: 0.6787 - acc: 0.7155 - val_loss: 0.5581 - val_acc: 0.7872
Epoch 4/10
249/249 [==============================] - 14s 58ms/step - loss: 0.5948 - acc: 0.7523 - val_loss: 0.5080 - val_acc: 0.8147
Epoch 5/10
249/249 [==============================] - 14s 56ms/step - loss: 0.5373 - acc: 0.7880 - val_loss: 0.5423 - val_acc: 0.7886
Epoch 6/10
249/249 [==============================] - 14s 56ms/step - loss: 0.5023 - acc: 0.8041 - val_loss: 0.5379 - val_acc: 0.7956
Epoch 7/10
249/249 [==============================] - 14s 55ms/step - loss: 0.4884 - acc: 0.8097 - val_loss: 0.4909 - val_acc: 0.8203
Epoch 8/10
249/249 [==============================] - 14s 55ms

In [174]:
from keras.models import load_model
model = load_model('/content/model_best_6.hdf5')

In [175]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [176]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_GRU.append(accu)

# 70 %

In [177]:
rate = 0.7

In [178]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [179]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [180]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [181]:
d= 60

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(GRU(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [182]:
# fit the model
mcp_save = ModelCheckpoint('model_best_7.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/10
290/290 [==============================] - 19s 58ms/step - loss: 0.9311 - acc: 0.5662 - val_loss: 0.8312 - val_acc: 0.6321
Epoch 2/10
290/290 [==============================] - 16s 57ms/step - loss: 0.7457 - acc: 0.6858 - val_loss: 0.7528 - val_acc: 0.7040
Epoch 3/10
290/290 [==============================] - 17s 58ms/step - loss: 0.6043 - acc: 0.7648 - val_loss: 0.5917 - val_acc: 0.7717
Epoch 4/10
290/290 [==============================] - 17s 58ms/step - loss: 0.5463 - acc: 0.7872 - val_loss: 0.5132 - val_acc: 0.8125
Epoch 5/10
290/290 [==============================] - 17s 60ms/step - loss: 0.5174 - acc: 0.8041 - val_loss: 0.5236 - val_acc: 0.8132
Epoch 6/10
290/290 [==============================] - 18s 62ms/step - loss: 0.4916 - acc: 0.8123 - val_loss: 0.4827 - val_acc: 0.8238
Epoch 7/10
290/290 [==============================] - 16s 57ms/step - loss: 0.4560 - acc: 0.8325 - val_loss: 0.4954 - val_acc: 0.8189
Epoch 8/10
290/290 [==============================] - 16s 56ms

In [183]:
from keras.models import load_model
model = load_model('/content/model_best_7.hdf5')

In [184]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [185]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_GRU.append(accu)

# 80 %

In [186]:
rate = 0.8

In [187]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [188]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [189]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [190]:
d= 60

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(GRU(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [191]:
# fit the model
mcp_save = ModelCheckpoint('model_best_8.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/10
331/331 [==============================] - 14s 37ms/step - loss: 0.9264 - acc: 0.5631 - val_loss: 0.7392 - val_acc: 0.6885
Epoch 2/10
331/331 [==============================] - 12s 36ms/step - loss: 0.7202 - acc: 0.6975 - val_loss: 0.6079 - val_acc: 0.7526
Epoch 3/10
331/331 [==============================] - 12s 35ms/step - loss: 0.5897 - acc: 0.7677 - val_loss: 0.5079 - val_acc: 0.8048
Epoch 4/10
331/331 [==============================] - 12s 35ms/step - loss: 0.5275 - acc: 0.8021 - val_loss: 0.5042 - val_acc: 0.8104
Epoch 5/10
331/331 [==============================] - 12s 35ms/step - loss: 0.4893 - acc: 0.8161 - val_loss: 0.4946 - val_acc: 0.8083
Epoch 6/10
331/331 [==============================] - 12s 35ms/step - loss: 0.4673 - acc: 0.8225 - val_loss: 0.4950 - val_acc: 0.8090
Epoch 7/10
331/331 [==============================] - 11s 35ms/step - loss: 0.4441 - acc: 0.8259 - val_loss: 0.5026 - val_acc: 0.8168
Epoch 8/10
331/331 [==============================] - 12s 36ms

In [192]:
from keras.models import load_model
model = load_model('/content/model_best_8.hdf5')

In [193]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [194]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_GRU.append(accu)

# 90 %

In [195]:
rate = 0.9

In [196]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [197]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [198]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [199]:
d= 60

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(GRU(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [200]:
# fit the model
mcp_save = ModelCheckpoint('model_best_9.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/10
373/373 [==============================] - 23s 55ms/step - loss: 0.9090 - acc: 0.5707 - val_loss: 0.7956 - val_acc: 0.6681
Epoch 2/10
373/373 [==============================] - 20s 53ms/step - loss: 0.6868 - acc: 0.7120 - val_loss: 0.5992 - val_acc: 0.7752
Epoch 3/10
373/373 [==============================] - 20s 53ms/step - loss: 0.5717 - acc: 0.7788 - val_loss: 0.5235 - val_acc: 0.8147
Epoch 4/10
373/373 [==============================] - 20s 53ms/step - loss: 0.5292 - acc: 0.7882 - val_loss: 0.4975 - val_acc: 0.8217
Epoch 5/10
373/373 [==============================] - 20s 54ms/step - loss: 0.5064 - acc: 0.8066 - val_loss: 0.4887 - val_acc: 0.8365
Epoch 6/10
373/373 [==============================] - 20s 53ms/step - loss: 0.4812 - acc: 0.8150 - val_loss: 0.5261 - val_acc: 0.8048
Epoch 7/10
373/373 [==============================] - 20s 54ms/step - loss: 0.4695 - acc: 0.8171 - val_loss: 0.4936 - val_acc: 0.8407
Epoch 8/10
373/373 [==============================] - 19s 52ms

In [201]:
from keras.models import load_model
model = load_model('/content/model_best_9.hdf5')

In [202]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [203]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_GRU.append(accu)

# 100 %

In [204]:
rate = 1

In [205]:
maxlen = 100 # cut point in the review
training_samples = int(len(text_all)*0.7*rate) # train size
validation_samples = len(text_all)-int(len(text_all)*0.7) # Validates size
max_words = 10000 # Considers only the top 10000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text_all)
sequences = tokenizer.texts_to_sequences(text_all)
word_index = tokenizer.word_index               

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(onehot_encoded)

indices = np.arange(data.shape[0]) # Splits data into training and validation set, but shuffles is, since samples are ordered: 
# all negatives first, then all positive
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
labels = np.asarray(labels).astype(np.float32)

X_train = data[:training_samples] 
y_train = labels[:training_samples]
X_test = data[training_samples:training_samples+validation_samples] 
y_test = labels[training_samples:training_samples+validation_samples]

In [206]:
# word embedding subsection
embeddings_index = {}

f = open("drive/MyDrive/glove.6B.100d.txt", encoding='utf-8') #added , encoding='utf-8'
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype="float32")
    embeddings_index[word] = coefs
f.close()

print("found %s word vectors." % len (embeddings_index))

found 400000 word vectors.


In [207]:
embedding_dim = 100 # GloVe contains 100-dimensional embedding vectors for 400.000 words

embedding_matrix = np.zeros((max_words, embedding_dim)) # embedding_matrix.shape (10000, 100)
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word) # embedding_vector.shape (100,)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector # Words not found in the mebedding index will all be zeros

In [208]:
d= 60

# built the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length = maxlen))
model.add(GRU(d, dropout = 0.3))
model.add(Dense(3, activation = "softmax"))
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

# compile
model.compile(optimizer = "rmsprop",
            loss = "categorical_crossentropy",
            metrics = ["acc"])

In [209]:
# fit the model
mcp_save = ModelCheckpoint('model_best_100.hdf5', save_best_only=True, monitor='val_loss', mode='min')
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test,y_test), callbacks=[mcp_save])

Epoch 1/10
414/414 [==============================] - 26s 57ms/step - loss: 0.8764 - acc: 0.5950 - val_loss: 1.0286 - val_acc: 0.6152
Epoch 2/10
414/414 [==============================] - 22s 53ms/step - loss: 0.6521 - acc: 0.7363 - val_loss: 0.5993 - val_acc: 0.7646
Epoch 3/10
414/414 [==============================] - 22s 53ms/step - loss: 0.5592 - acc: 0.7735 - val_loss: 0.5168 - val_acc: 0.8013
Epoch 4/10
414/414 [==============================] - 22s 53ms/step - loss: 0.5136 - acc: 0.8043 - val_loss: 0.5133 - val_acc: 0.8013
Epoch 5/10
414/414 [==============================] - 22s 53ms/step - loss: 0.4812 - acc: 0.8173 - val_loss: 0.4987 - val_acc: 0.8196
Epoch 6/10
414/414 [==============================] - 22s 52ms/step - loss: 0.4654 - acc: 0.8206 - val_loss: 0.4946 - val_acc: 0.8189
Epoch 7/10
414/414 [==============================] - 22s 52ms/step - loss: 0.4506 - acc: 0.8248 - val_loss: 0.4915 - val_acc: 0.8224
Epoch 8/10
414/414 [==============================] - 22s 53ms

In [210]:
from keras.models import load_model
model = load_model('/content/model_best_100.hdf5')

In [211]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_test_1 = np.argmax(y_test, axis=1)

In [212]:
from sklearn.metrics import classification_report

accu = classification_report(y_test_1, y_pred, output_dict = True)['accuracy']
history_GRU.append(accu)

# save history

In [213]:
import pickle
with open('GRU_history.pkl', 'wb') as f:
  pickle.dump(history_GRU, f)